# Least Squares Regression of the Logistic and Gompertz Models

---

The logistic and Gompertz can be linearized by taking the logarithm of the response variable. This is a common technique used in regression analysis to linearize non-linear models. 